# Intermediate Python (1): Classes

by [Yao-Yuan Mao](https://yymao.github.io)

Write 2-3 compatible code: http://python-future.org/compatible_idioms.html

In [1]:
from __future__ import division, print_function

# `conda install future` OR `pip install future` 
from builtins import *

## Nomenclature: object, class, instance, attribute, method

https://docs.python.org/2.7/tutorial/classes.html OR https://docs.python.org/3.6/tutorial/classes.html

In [2]:
class Human(object):
    def eat(self, food):
        print('I am eating {}'.format(food))

In [3]:
# create an instance `Yao` from the class `Human`
Yao = Human() 

# access method `eat`
Yao.eat('an apple')

# set and get attribute `name`
Yao.name = "Yao"
print(Yao.name)

I am eating an apple
Yao


## How and when to use a class in Python
- Maintain persistent states
- Enhance readability (abstraction, encapsulation)
- Enhance extensibility (polymorphism, inheritance)

Take a look at an example

In [4]:
# what we usually do 

def load(fn):
    pass

def process(im):
    return True

def plot(im):
    pass

for fn in ['a', 'b','c']:
    im = load(fn)
    im_processed = process(im)
    plot(im_processed)

In [5]:
# what we can do:

class Image(object):
    def __init__(self, fname):
        self.fname = fname
        self.im = load(fname)
        self.im_processed = None
        
    def process(self):
        self.im_processed = process(self.im)
        
    def plot(self):
        # to prevent plotting if the image has not been processed
        if self.im_processed is None:
            raise ValueError
        plot(im_processed)
        
for fn in ['a', 'b','c']:
    im = Image(fn)
    im.process()
    im.plot()
    print(im.fname)

a
b
c


Another example

In [6]:
import math

def NFWProfile(r, rs, rhos):
    x = r/rs
    return rhos/(x*(1.0+x)**2.0)

print(NFWProfile(8.0, 20.0, 0.8))
print(NFWProfile(8.0, 10.0, 2.0))

1.02040816327
0.771604938272


In [7]:
params1 = {'rs':20.0, 'rhos':0.8}
NFWProfile(8.0, **params1)

1.0204081632653064

In [8]:
def GeneralizedNFWProfile(r, rs, rhos, gamma):
    x = r/rs
    return rhos/(x*(1.0+x)**gamma)

In [9]:
# does not work
GeneralizedNFWProfile(8.0, **params1)

TypeError: GeneralizedNFWProfile() takes exactly 4 arguments (3 given)

In [10]:
# now this work
params1 = {'rs':20.0, 'rhos':0.8, 'gamma':2.0}
GeneralizedNFWProfile(8.0, **params1)

1.0204081632653064

In [11]:
# but this does not:
NFWProfile(8.0, **params1)

TypeError: NFWProfile() got an unexpected keyword argument 'gamma'

In [12]:
# what you can do

class Profile(object):
    def __init__(self, func, **kwargs):
        self.func = func
        self.kwargs = kwargs
        
    def call(self, r):
        return self.func(r, **self.kwargs)
        
p1 = Profile(NFWProfile, rs=20.0, rhos=0.8)
p1.call(8.0)

1.0204081632653064

## Things that start with underscore(s), like "\__init\__", are

In [13]:
class Human(object):
    def __init__(self, name):
        # call when the instance is initialized (but not when it was created!)
        print('haha')
        self.name = name
        
    def eat(self, food):
        print('I am eating {}'.format(food))
        
    def __add__(self, other):
        # tell "+" what to do
        print(self.name, other.name)
        
    def __iter__(self):
        # tell "for ... in" what to do
        print('this is __iter__')
        return iter([])
    
    def __call__(self, x):
        # tell "()" what to do
        return x+1
    
    def __getitem__(self, x):
        return self.name[x]
    
    def __len__(self):
        # tell "len()" what to do
        return len(self.name)

In [14]:
Yao = Human('Yao')
Andrew = Human('Andrew')

haha
haha


In [15]:
Yao + Andrew

Yao Andrew


In [16]:
Yao(2)

3

In [17]:
Yao[1]

'a'

In [18]:
len(Yao)

3

See a full list at:
https://docs.python.org/2/reference/datamodel.html#special-method-names

## hasattr(), setattr(), and getattr()

In [19]:
def addone(x):
    return x+1

y = 1

print(hasattr(addone, '_call__'))
print(hasattr(y, '_call__'))
print(callable(addone))
print(hasattr([1,2,3], '__iter__'))
print(hasattr('abc', '__iter__'))

False
False
True
True
False


In [20]:
# this is identical to Yao.school = 'Pitt'
setattr(Yao, 'school', 'Pitt')
print(Yao.school)

Pitt


In [21]:
# set and get attributes with variables

d = dict(school='Pitt', position='Postdoc', advisor='Andrew')

for k, v in d.iteritems():
    setattr(Yao, k, v)
    
for k in d:
    print(getattr(Yao, k))

Postdoc
Pitt
Andrew


## @staticmethod and @classmethod

In [22]:
class Human(object):
    
    # this is a class attribute, not an instance attribute
    species = 'Homo sapiens'
    
    def __init__(self, name):
        self.name = name
        
    def evolve(self):
        self.species = 'Homo posterus'


Yao = Human('Yao')
Andrew = Human('Andrew')

# Yao (as an instance) has evolved, but not the Human class!
Yao.evolve()

print(Yao.name, Yao.species)
print(Andrew.name, Andrew.species)
print(Yao.name, Yao.__class__.species)

Yao Homo posterus
Andrew Homo sapiens
Yao Homo sapiens


In [23]:
class Human(object):
    species = 'Homo sapiens'
    
    def __init__(self, name):
        self.name = name
    
    # with @classmethod, the first argument that will be passed to `evolve` will be the `Human` class, 
    # NOT the instance!
    @classmethod
    def evolve(cls):
        cls.species = 'Homo posterus'


Yao = Human('Yao')
Andrew = Human('Andrew')

Yao.evolve()

print(Yao.name, Yao.species)
print(Andrew.name, Andrew.species)
print(Yao.name, Yao.__class__.species)

Yao Homo posterus
Andrew Homo posterus
Yao Homo posterus


In [24]:
class Cosmology(object):
    def __init__(self, Omega_M):
        self.Omega_M = Omega_M
        
    @staticmethod
    def a2z(a):
        return 1.0/a - 1.0    

In [25]:
cosmo = Cosmology(1.0)
print(Cosmology.a2z(1.0))
print(cosmo.a2z(1.0))

0.0
0.0


While `@classmethod` and `@staticmethod` are not commonly used. This part is to help you distinquish the difference between a class and its instances.

To summarize: 

- a method by default, when called, uses the calling instance as the first argument
  (hence you cannot do `Human.eat()` in our above example, since `Human` is not an instance)
  
- if a method is decorated by `@classmethod`, it will use the calling **class** as the first argument

- if a method is decorated by `@staticmethod`, it will not use the calling instance nor class as the first argument

## Class inheritance

In [26]:
class Human(object):
    def __init__(self, name):
        self.name = name
        
    def eat(self, food):
        print('{} is eating {}'.format(self.name, food))
        
    def drink(self, liquid):
        print('{} is drinking {}'.format(self.name, liquid))

        
class Astrophysicist(Human):
    # add new methods
    def write_paper(self):
        print('I am writing papers')
        
    # overwrite methods
    def drink(self, liquid):
        if liquid != 'coffee':
            print('I only drink coffee')
        else:
            # use super to call the original function in the parent class
            super(Astrophysicist, self).drink(liquid)

In [27]:
Yao = Astrophysicist('Yao')

# parent methods still work
Yao.eat('an apple')

Yao.write_paper()

Yao.drink('juice')
Yao.drink('coffee')

Yao is eating an apple
I am writing papers
I only drink coffee
Yao is drinking coffee


## type(), isinstance(), issubclass()

In [28]:
# everything in python is an object (and hence has a type!)

print(type(Yao))
print(type(123))
print(type('abc'))
print(type(str))
print(type(type))

<class '__main__.Astrophysicist'>
<type 'int'>
<type 'str'>
<class 'future.types.newstr.BaseNewStr'>
<type 'type'>


In [29]:
print(issubclass(type(Yao), Human))

True


In [30]:
print(issubclass(type('abc'), str))
print(issubclass(type(u'abc'), str))
print(issubclass(type(u'abc'), basestring))

False
False
True


In [31]:
print(isinstance([1,2,3], list))
print(isinstance((1,2,3), list)) # (1,2,3) is a tuple

# better way to check if iterable is
print(hasattr((1,2,3), '__iter__'))

True
False
True
